# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/DBTL7/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/DBTL7/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [23]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [24]:
CYCLE = 3.1

user_params = {
    'media_file': f'../flaviolin data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [25]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.002768,20.0,0.295690,11.653411,0.015110,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05
B1,40.0,4.0,0.002768,20.0,0.295690,11.653411,0.015110,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05
C1,40.0,4.0,0.002768,20.0,0.295690,11.653411,0.015110,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05
D1,40.0,4.0,0.002039,20.0,0.306612,11.755484,0.013897,13.529082,0.460584,517.597679,0.000043,0.004579,0.002872,0.012345,0.001194,0.05
E1,40.0,4.0,0.002039,20.0,0.306612,11.755484,0.013897,13.529082,0.460584,517.597679,0.000043,0.004579,0.002872,0.012345,0.001194,0.05


## Create Line Description

In [26]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [27]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F3    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [28]:
column_order = True if df.index[1][0] == 'B' else False

In [29]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C3.1_WA1_C1-R1
C3.1_WA1_C1-R2
C3.1_WA1_C1-R3
C3.1_WD1_F1-R1
C3.1_WD1_F1-R2
C3.1_WD1_F1-R3
C3.1_WA2_C2-R1
C3.1_WA2_C2-R2
C3.1_WA2_C2-R3
C3.1_WD2_F2-R1
C3.1_WD2_F2-R2
C3.1_WD2_F2-R3
C3.1_WA3_C3-R1
C3.1_WA3_C3-R2
C3.1_WA3_C3-R3
C3.1_WD3_F3-R1
C3.1_WD3_F3-R2
C3.1_WD3_F3-R3
C3.1_WA4_C4-R1
C3.1_WA4_C4-R2
C3.1_WA4_C4-R3
C3.1_WD4_F4-R1
C3.1_WD4_F4-R2
C3.1_WD4_F4-R3
C3.1_WA5_C5-R1
C3.1_WA5_C5-R2
C3.1_WA5_C5-R3
C3.1_WD5_F5-R1
C3.1_WD5_F5-R2
C3.1_WD5_F5-R3
C3.1_WA6_C6-R1
C3.1_WA6_C6-R2
C3.1_WA6_C6-R3
C3.1_WD6_F6-R1
C3.1_WD6_F6-R2
C3.1_WD6_F6-R3
C3.1_WA7_C7-R1
C3.1_WA7_C7-R2
C3.1_WA7_C7-R3
C3.1_WD7_F7-R1
C3.1_WD7_F7-R2
C3.1_WD7_F7-R3
C3.1_WA8_C8-R1
C3.1_WA8_C8-R2
C3.1_WA8_C8-R3
C3.1_WD8_F8-R1
C3.1_WD8_F8-R2
C3.1_WD8_F8-R3


In [30]:
df.head(5)

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.002768,20.0,0.295690,11.653411,0.015110,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C3.1_WA1_C1-R1
B1,40.0,4.0,0.002768,20.0,0.295690,11.653411,0.015110,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C3.1_WA1_C1-R2
C1,40.0,4.0,0.002768,20.0,0.295690,11.653411,0.015110,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C3.1_WA1_C1-R3
D1,40.0,4.0,0.002039,20.0,0.306612,11.755484,0.013897,13.529082,0.460584,517.597679,0.000043,0.004579,0.002872,0.012345,0.001194,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C3.1_WD1_F1-R1
E1,40.0,4.0,0.002039,20.0,0.306612,11.755484,0.013897,13.529082,0.460584,517.597679,0.000043,0.004579,0.002872,0.012345,0.001194,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C3.1_WD1_F1-R2


## Process measurement files

Read measurements file:

In [31]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

# df_600

In [32]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.6302,0.5756,0.5786,0.5759,0.5338,0.5534,0.5484,0.5727,0.1209,0.1203,0.1192,0.1153
B,0.5994,0.5587,0.5569,0.5837,0.6701,0.6592,0.6775,0.5586,0.1208,0.1173,0.1168,0.1113
C,0.5894,0.5738,0.5780,0.6045,0.1397,0.5721,0.7069,0.5843,0.1131,0.1129,0.1092,0.1070
D,0.5621,0.5487,0.5547,0.6289,0.6509,0.6308,0.5303,0.3708,0.1052,0.1057,0.1034,0.0994
E,0.5793,0.5800,0.5792,0.5422,0.6701,0.6681,0.6876,0.3962,0.1001,0.0999,0.0984,0.0937
F,0.5790,0.5835,0.5886,0.5396,0.5709,0.5341,0.5489,0.3901,0.0967,0.0967,0.0938,0.0904
G,0.0893,0.0924,0.0943,0.0949,0.0959,0.0942,0.0939,0.0974,0.0958,0.0944,0.0915,0.0884
H,0.0903,0.0924,0.0945,0.0932,0.0927,0.0948,0.0924,0.0923,0.0931,0.0971,0.0909,0.0893


### Process OD600

Normalize the data to the control well A9 value (water content):

In [33]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.7337,0.5735,0.6796,0.6332,0.8774,0.9875,0.8713,0.7177,0.0000,0.0088,0.0093,0.0083
B,0.6245,0.4985,0.5544,0.6003,0.7672,0.8557,0.7709,0.7489,0.0060,0.0051,0.0057,0.0044
C,0.6978,0.5597,0.4411,0.6442,0.0068,0.2983,0.7802,0.7389,-0.0007,0.0006,-0.0006,-0.0004
D,0.6816,0.5119,0.5471,0.6655,0.7859,0.7174,0.8122,0.0171,-0.0059,-0.0056,-0.0056,-0.0069
E,0.7649,0.5864,0.5388,0.5121,0.7297,0.8656,0.8267,0.0120,-0.0082,-0.0086,-0.0106,-0.0116
F,0.7129,0.6854,0.7273,0.7511,0.8661,0.8861,0.9139,0.0099,-0.0139,-0.0136,-0.0144,-0.0142
G,-0.0148,-0.0147,-0.0143,-0.0144,-0.0146,-0.0148,-0.0151,-0.0149,-0.0144,-0.0149,-0.0152,-0.0153
H,-0.0135,-0.0134,-0.0139,-0.0141,-0.0143,-0.0144,-0.0143,-0.0143,-0.0139,-0.0139,-0.0141,-0.0143


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [34]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,7.337,5.735,6.796,6.332,8.774,9.875,8.713,7.177
B,6.245,4.985,5.544,6.003,7.672,8.557,7.709,7.489
C,6.978,5.597,4.411,6.442,0.068,2.983,7.802,7.389
D,6.816,5.119,5.471,6.655,7.859,7.174,8.122,0.171
E,7.649,5.864,5.388,5.121,7.297,8.656,8.267,0.120
F,7.129,6.854,7.273,7.511,8.661,8.861,9.139,0.099


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [35]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.5093,0.4547,0.4577,0.4550,0.4129,0.4325,0.4275,0.4518
B,0.4785,0.4378,0.4360,0.4628,0.5492,0.5383,0.5566,0.4377
C,0.4685,0.4529,0.4571,0.4836,0.0188,0.4512,0.5860,0.4634
D,0.4412,0.4278,0.4338,0.5080,0.5300,0.5099,0.4094,0.2499
E,0.4584,0.4591,0.4583,0.4213,0.5492,0.5472,0.5667,0.2753
F,0.4581,0.4626,0.4677,0.4187,0.4500,0.4132,0.4280,0.2692


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [36]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    # df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    # df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

# df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

# df_600.index = df_600['Well']
df_340.index = df_340['Well']

# df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

# df_600.head()

Add measurements to the main dataframe:

In [37]:
# measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
# df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.002768,20.0,0.29569,11.653411,0.01511,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C3.1_WA1_C1-R1,0.5093
B1,40.0,4.0,0.002768,20.0,0.29569,11.653411,0.01511,13.297119,0.451467,427.961444,0.000028,0.003668,0.004136,0.012464,0.001219,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C3.1_WA1_C1-R2,0.4785


## Create EDD Experiment Description File

In [38]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [39]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [40]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [41]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
# df['Measurement Type'] = 'Optical Density'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD600]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [42]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [43]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)